In [7]:
import ast
import boto3
import firebase_admin
import json
import math
import numpy as np
import os
import pandas as pd
import requests
import time
import torch
import torch.nn.functional as F

from botocore.exceptions import ClientError
from decimal import Decimal
from dotenv import load_dotenv
from firebase_admin import credentials, firestore
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
#from torch.nn.functional import cosine_similarity

In [2]:
# uploading the environment variables and get the API key
load_dotenv()
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")

In [35]:
# Genres database, local by now
df_genres = pd.read_csv(r'generos.csv', sep=',')


### Content Data from FireBase (remains missing the conextion to firebase collection)

In [8]:
# Start Firebase if it's not done
if not firebase_admin._apps:
    cred_path = r'C:\Users\Agustín\OneDrive\Formación\2. Practicas\Data Scientist & ML Engineer\bubbo-dfba0-firebase-adminsdk-fbsvc-79dc4511e7.json'  # Asegúrate de que la ruta sea correcta
    cred = credentials.Certificate(cred_path)
    firebase_admin.initialize_app(cred)

In [36]:
# Firestore conexion and db collection name
db = firestore.client()
collection_ref = db.collection('Data_Clean')
# to get it all
docs = collection_ref.stream()
# documents to dictionaries
data = [{**doc.to_dict(), 'id': doc.id} for doc in docs]
df = pd.DataFrame(data)

In [41]:
filtered_data = df
filtered_data = filtered_data.replace("",pd.NA)
filtered_data = filtered_data.dropna()
filtered_data = filtered_data.drop_duplicates()
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 141207 entries, 0 to 141441
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   CleanTitle    141207 non-null  object
 1   PlatformName  141207 non-null  object
 2   ID            141207 non-null  object
 3   Genre         141207 non-null  object
 4   Type          141207 non-null  object
 5   Synopsis      141207 non-null  object
 6   Cast          141207 non-null  object
 7   Directors     141207 non-null  object
 8   id            141207 non-null  object
dtypes: object(9)
memory usage: 10.8+ MB


### User Preferences from DynamoDB

In [51]:
# To get the info from DynamoDB, user preferences
CONFIG = {
    'aws': {
        'access_key': AWS_ACCESS_KEY,
        'secret_key': AWS_SECRET_KEY,
        'region': 'eu-west-3',
        'table': 'User-7kkcm5dn2rb77hst5nh7gbdisa-staging'
    },
    'columns': ['userId', 'favoriteMoviesIds', 'favoriteGenresIds', 'favoriteSeriesIds'],
}

# conexion
session = boto3.Session(
    aws_access_key_id=CONFIG['aws']['access_key'],
    aws_secret_access_key=CONFIG['aws']['secret_key'],
    region_name=CONFIG['aws']['region']
)

table = session.resource('dynamodb').Table(CONFIG['aws']['table'])

# Values to String
def _process_value(value):
    if isinstance(value, Decimal):
        return str(int(value))
    return str(value)

# Retrive info from DynamoDB and gets a DataFrame
def fetch_preferences():
    try:
        items = []
        start_key = None

        while True:
            # scan with defined 'columns'  in previous 'CONFIG'
            scan_params = {
                'ProjectionExpression': ', '.join(CONFIG['columns'])
            }
            if start_key:
                scan_params['ExclusiveStartKey'] = start_key

            response = table.scan(**scan_params)
            items.extend(response.get('Items', []))

            # check for next pages
            start_key = response.get('LastEvaluatedKey')
            if not start_key:
                break

        # data extracted processing
        processed_data = [{
            'userId': _process_value(item.get('userId', '')),
            'favoriteMoviesIds': ';'.join(map(_process_value, item.get('favoriteMoviesIds', []))),
            'favoriteGenresIds': ';'.join(map(_process_value, item.get('favoriteGenresIds', []))),
            'favoriteSeriesIds': ';'.join(map(_process_value, item.get('favoriteSeriesIds', [])))
        } for item in items]

        df = pd.DataFrame(processed_data)
        return df

    except ClientError as e:
        print(f"Error al conectar con DynamoDB: {e}")
        return pd.DataFrame()

# calling function to get the df
user_pref = fetch_preferences()


In [52]:
# limpio el dataframe dejando solo users con genero, movie_favs y tvshow_favs
user_pref = user_pref[user_pref['userId'].str.len()==36]
user_pref = user_pref.replace("",pd.NA)
user_pref = user_pref.dropna()
user_pref.reset_index(inplace=True,drop=True)
print(f'Duplicates: {user_pref.duplicated().sum()}')
user_pref.info()

Duplicates: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7431 entries, 0 to 7430
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   userId             7431 non-null   object
 1   favoriteMoviesIds  7431 non-null   object
 2   favoriteGenresIds  7431 non-null   object
 3   favoriteSeriesIds  7431 non-null   object
dtypes: object(4)
memory usage: 232.3+ KB


In [53]:
user_pref.head(2)

,userId,favoriteMoviesIds,favoriteGenresIds,favoriteSeriesIds
0,d17900ae-a001-70a3-d3bc-9463452af02c,155;293660;24428;299536;118340,28;10759;12;16;10762;80;99;10764;10763;18;35;1...,1399;71446;1402;76479;100088
1,c1f9d00e-6001-70e0-28b6-9fcaee14754c,24428;299536;118340;150540;283995,16;10762;14;10765;878;12;10759;10751,66732;93405;456;119051;65334


In [ ]:
# celda para producir no terminada
# Hay un error en los generos que antes los traia de un csv y ahora lo traigo de cada contenido en firebase, corregir antes de dar finalizado


# convertir los valores en listas para expandirlos con explode
user_pref['favoriteGenresIds'] = user_pref['favoriteGenresIds'].apply(lambda x: x.split(';'))
user_pref['favoriteMoviesIds'] = user_pref['favoriteMoviesIds'].apply(lambda x: x.split(';'))
user_pref['favoriteSeriesIds'] = user_pref['favoriteSeriesIds'].apply(lambda x: x.split(';'))

# expandir preferencias de favoriteMoviesIds, favoriteGenresIds, y favoriteSeriesIds por userId
user_fav_genres = user_pref[['userId','favoriteGenresIds']].explode('favoriteGenresIds')
user_fav_movies = user_pref[['userId','favoriteMoviesIds']].explode('favoriteMoviesIds')
user_fav_series = user_pref[['userId','favoriteSeriesIds']].explode('favoriteSeriesIds')


# merge para traerme los CleanTitle, Synopsis, 'Genre'
user_fav_genres = user_fav_genres.merge(df_genres[['genero_id','genero_name']], left_on='favoriteGenresIds', right_on='genero_id') #### ACA IRIA CONCAT
user_fav_movies = user_fav_movies.merge(filtered_data[['ID','CleanTitle']], left_on='favoriteMoviesIds', right_on='ID')
user_fav_series = user_fav_series.merge(filtered_data[['ID','CleanTitle']], left_on='favoriteSeriesIds', right_on='ID')

user_fav_genres = user_fav_genres.drop(columns='ID')
user_fav_movies = user_fav_movies.drop(columns='ID')
user_fav_series = user_fav_series.drop(columns='ID')
'''

# reAGRUPO por userId para que me queden las listas CleanTitle, Synopsis, 'Genre' por user segun sus favoriteMoviesIds, favoriteGenresIds, y favoriteSeriesIds por userId
user_pref_prueba_genres = user_fav_genres.groupby('userId')[['favoriteGenresIds','Genre']].agg(list).reset_index()
user_pref_prueba_movies = user_fav_movies.groupby('userId')[['favoriteMoviesIds','CleanTitle']].agg(list).reset_index()
user_pref_prueba_series = user_fav_series.groupby('userId')[['favoriteSeriesIds','CleanTitle']].agg(list).reset_index()


user_fav_genres

### >>>>>>>>>>>> TERMINAR DE HACER LO MISMO CON GENERO Y SERIES'''

ValueError: You are trying to merge on object and int64 columns for key 'favoriteGenresIds'. If you wish to proceed you should use pd.concat

In [55]:
user_fav_genres.head(2)

,userId,favoriteGenresIds
0,d17900ae-a001-70a3-d3bc-9463452af02c,28
0,d17900ae-a001-70a3-d3bc-9463452af02c,10759


In [56]:
# Celda para borrar
df_genres.head(2)

,genero_id,genero_name
0,28,Acción
1,12,Aventura


In [ ]:
#celda para borrar
filtered_data.head(2)

### Sentences for vectorize from 'filtered_data' / AHORA DESDE FIREBASE

In [ ]:
# Making the sentences to embed 
filtered_data['sentences_to_embed'] = (
    filtered_data.CleanTitle.fillna('') +
    filtered_data.Synopsis.fillna('') +
    filtered_data.Genre.fillna('').apply(
        lambda x: ', '.join(ast.literal_eval(x)) if x.startswith('[') and x.endswith(']') else x
    )
)

# Formatting as str-list to send to the model
sentences_from_filtered_data = filtered_data.sentences_to_embed.dropna().astype(str).tolist()


In [ ]:
# Since the model on Hugging Face processes only requests that can be completed within 60 seconds, we need to divide the sentences into batches.
def split_into_batches(sentences, batch_size):
    return [sentences[i:i + batch_size] for i in range(0, len(sentences), batch_size)]

# After trying with different values, we've reach the maximum batch size to get response succesfully
batches = split_into_batches(sentences_from_filtered_data, 100)

In [ ]:
# Check key availability
if HUGGINGFACE_API_KEY is None:
    print("Error: No se encontró la clave de API de Hugging Face.")
else:
    print("Clave de API cargada correctamente.")

# Model URL
API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

# API header and key
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}  

# Function to get embeddings from Hugging Face API
def get_embeddings_from_api(sentences):
    url = API_URL
    payload = {"inputs": sentences}
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None
    

# Sequence Batch Processing Process
all_embeddings_from_filtered_data = []
for batch in batches:
    print(f"Processing batch with {len(batch)} sentences...")
    embeddings = get_embeddings_from_api(batch)
    if embeddings:
        all_embeddings_from_filtered_data.extend(embeddings)

print("Embeddings processed successfully:")
print(all_embeddings_from_filtered_data[:2])  

### Esta sección es para agregar los titulos de los generos, y los str correspondientes

### Sentences for vectorize from 'user_preferences' / AHORA DESDE DYNAMODB

In [ ]:
# Sentences we want to be embedded from user_preferences
user_pref['sentences_to_embed'] = (user_pref.TitulosPeliculas.fillna('') +
                                   user_pref.TitulosSeries.fillna('') + 
                                   user_pref.GenerosFavoritos.fillna('') +
                                   user_pref.DetallesPeliculas.fillna('') +
                                   user_pref.DetallesSeries.fillna(''))

# Formatting as str-list to send to the model
sentences_from_user_pref = user_pref.sentences_to_embed.dropna().astype(str).tolist()

# We split the sentences in batches as we did previously with filtered_data
batches_user_pref = split_into_batches(sentences_from_user_pref, 100)

In [ ]:
# Sequence Batch Processing Process
all_embeddings_from_user_pref = []
for batch in batches_user_pref:
    print(f"Processing batch with {len(batch)} sentences...")
    embeddings = get_embeddings_from_api(batch)
    if embeddings:
        all_embeddings_from_user_pref.extend(embeddings)

print("Embeddings processed successfully:")
print(all_embeddings_from_user_pref[:2])  

### Similarities

In [ ]:
# We have to convert the embeddings list to numpy arrays in order to calculate cosine similarities wiht sklearn
all_embeddings_from_user_pref_array = np.array(all_embeddings_from_user_pref)
all_embeddings_from_filtered_data_array = np.array(all_embeddings_from_filtered_data)

user_for_example = 1

# Taking first user as example to calculate the cosine_similarity
user_embedding_example = all_embeddings_from_user_pref_array[user_for_example].reshape(1, -1)  # Asegurar forma correcta para cosine_similarity

# To calculate similarity between the user example embeding and the whole content from filtered data
content_similarities = cosine_similarity(user_embedding_example, all_embeddings_from_filtered_data_array).flatten()

# Sort indexes by similarity
most_similar_indexes = content_similarities.argsort()[::-1]

# Top-10
topten_most_similar_indexes = most_similar_indexes[:10]

In [ ]:
# To display the most similar indexes and their similarity scores
print("Most similar indexes:", most_similar_indexes[:10])
print("Highest similarities:", content_similarities[most_similar_indexes[:10]])

In [ ]:
# search most_similar_indexes, and preferences, and get recommendations
user_id = user_pref.loc[user_for_example]['userId']
movies_preferred = user_pref[user_pref['userId']==user_id]['TitulosPeliculas']
series_preferred = user_pref[user_pref['userId']==user_id]['TitulosSeries']

# Displaying Preferences & Recommendations
print(f'''User {user_id} Preferences:
      ''')
print(f'''Movies preference:''')
for movie in movies_preferred.iloc[0].split(';'):
    print(f'      {movie.strip()}')

print(f'''TV Shows preference:''')
for series in series_preferred.iloc[0].split(';'):
    print(f'      {series.strip()}')

print(f'''Recomendations for user: {user_id}
      ''')

recomendations_user = filtered_data.loc[topten_most_similar_indexes]['CleanTitle']
for recommendation in recomendations_user:
    print(f'      {recommendation}')
